# Setting up Data

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

## Creating a Bundle

In [5]:
# https://github.com/aspromatis/zipline_bundle/blob/master/prepare_bundle.py
# Assuming crypto
import csv
import numpy as np
import pandas as pd
import os

def format_bundle(indir, outdir):
       
    count = 0
    for f in os.listdir(indir): # For Production
    # for f in ['AAPL.csv']:  # For Testing
        if f[-4:] != '.csv':
            continue
        
        df = pd.read_csv(os.path.join(indir,f), index_col='Date')
        
        # trading days will be the same 
        #df = df.reindex(trading_days)

        # Export it in the csvdir format needed for the zipline bundle ingestion process
        df.reset_index(inplace=True)
        
        # drop adj close column
        df.drop('Adj Close', axis=1, inplace=True)
        
        # Check if there is there is any dividend, if not make it zero
        if not 'dividend' in df.columns:
            df['dividend'] = 0.0
        
        # Assume the only columns that exist in the .csv are [Date, Open, High, Low, Close, Adj Close, Volume] in that order
        #df = df[['date', 'open_adj', 'high_adj', 'low_adj', 'close_adj', 'volume_adj', 'dividend']]
        
        df['dividend'].fillna(0.00, inplace=True)
        df['ratio'] = 1  # There are no splits
        
        old_colnames = df.columns
        
        df.rename(columns={old_colnames[0]: 'date',
                           old_colnames[1]: 'open',
                           old_colnames[2]: 'high',
                           old_colnames[3]: 'low',
                           old_colnames[4]: 'close',
                           old_colnames[5]: 'volume'
                           }, inplace=True)
        
        # Round the numbers in the dataframe
        # won't work very well for small-cap coins?
        # could put off rounding?
        #df = df.round({'open':2,
        #          'high':2,
        #          'low':2,
        #          'close':2,
        #          'volume':1,
        #          'dividend':2})

        df.to_csv(os.path.join(outdir, f), index=False)

        count += 1
        
    return ('{} files were adjusted'.format(count))

format_bundle("../data/instruments", "../data/daily")

'7 files were adjusted'